In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
!pip install transformers trl datasets bitsandbytes peft qwen-vl-utils accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.2/376.2 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 111.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
#google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
import torch
import random
import warnings
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
from tqdm import tqdm
from transformers import Qwen2VLForConditionalGeneration, Qwen2VLProcessor, BitsAndBytesConfig
from transformers import AutoTokenizer,AutoModelForCausalLM
from peft import LoraConfig, get_peft_model
from trl import SFTConfig, SFTTrainer
from qwen_vl_utils import process_vision_info
from sklearn.model_selection import train_test_split

In [5]:
# 환경 설정
warnings.filterwarnings("ignore")
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()
print("✅ Using device:", device)

✅ Using device: cuda


In [6]:
# 시드 고정
def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything()

In [ ]:
os.chdir('/path/to/lib/Qwen2-VL-2B-MMStar')

# Load Datasets And Preprocessing

In [ ]:
MMStar_dataset = load_dataset("JHGondori/K-MMStar_English_Private")

In [9]:
system_message = """You are a multimodal assistant specialized in answering visual multiple-choice questions.
Given an image and a related question with options A, B, C, and D, select the single best answer based solely on the visual content and question.
Answer only with one of: A, B, C, or D. Do not provide explanations unless explicitly requested."""

In [10]:
def format_MMStardata(sample):
    return [
        {
            "role": "system",
            "content": [{"type": "text", "text": system_message}],
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "image": sample["image"]
                },
                {
                    "type": "text",
                    "text": f"Question: {sample['question_en']}\n" +
                    "\nAnswer:"
                },
            ],
        },
        {
            "role": "assistant",
            "content": [{"type": "text", "text": sample["answer"]}],
        }
    ]

In [11]:
train_dataset = [format_MMStardata(sample) for sample in MMStar_dataset['val']]

## local train data

In [12]:
def format_localdata(sample):
    return [
        {
            "role": "system",
            "content": [{"type": "text", "text": system_message}],
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "image": "." + sample["img_path"]
                },
                {
                    "type": "text",
                    "text": f"Question: {sample['Question']}\n" +
                    "\n".join([f"{chr(65+i)}. {choice}" for i, choice in enumerate([sample[c] for c in ['A', 'B', 'C', 'D']])]) +
                    "\nAnswer:"
                },
            ],
        },
        {
            "role": "assistant",
            "content": [{"type": "text", "text": sample["answer"]}],
        }
    ]

In [13]:
localtrain_dataset = load_dataset("csv", data_files = "../train.csv")

Generating train split: 0 examples [00:00, ? examples/s]

In [14]:
train_dataset = train_dataset + [format_localdata(sample) for sample in localtrain_dataset['train']]

In [15]:
train_dataset[1498]

[{'role': 'system',
  'content': [{'type': 'text',
    'text': 'You are a multimodal assistant specialized in answering visual multiple-choice questions.\nGiven an image and a related question with options A, B, C, and D, select the single best answer based solely on the visual content and question.\nAnswer only with one of: A, B, C, or D. Do not provide explanations unless explicitly requested.'}]},
 {'role': 'user',
  'content': [{'type': 'image',
    'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=518x80>},
   {'type': 'text',
    'text': 'Question: Consider the magnetic force between each pair of magnets. Which of the following statements is true?\n\nA: The magnetic force is weaker in Pair 2.\nB: The strength of the magnetic force is the same in both pairs.\nC: The magnetic force is weaker in Pair 1.\nD: None.\n\nAnswer:'}]},
 {'role': 'assistant', 'content': [{'type': 'text', 'text': 'C'}]}]

In [16]:
train_dataset = train_dataset[780 : 1170] #50% ~ 75%

In [17]:
train_dataset, val_dataset = train_test_split(train_dataset, test_size = 0.1)

In [18]:
len(val_dataset)

39

# Model finetuning

In [19]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [20]:
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.05,
    r=8,
    bias="none",
    target_modules=["q_proj", "v_proj"],
    task_type="CAUSAL_LM",
)

In [21]:
model_id = "Qwen/Qwen2-VL-2B-Instruct"

In [22]:
model = Qwen2VLForConditionalGeneration.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config
)
processor = Qwen2VLProcessor.from_pretrained(model_id)

config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/429M [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.


chat_template.json: 0.00B [00:00, ?B/s]

In [23]:
adapter_path = "./qwen2-VL-2B-instruct-trl-VQA-MMStar2/checkpoint-220"
model.load_adapter(adapter_path)

In [24]:
peft_model = get_peft_model(model, peft_config)
peft_model
peft_model.print_trainable_parameters()

trainable params: 1,089,536 || all params: 2,210,075,136 || trainable%: 0.0493


In [25]:
model.to(device)

Qwen2VLForConditionalGeneration(
  (model): Qwen2VLModel(
    (visual): Qwen2VisionTransformerPretrainedModel(
      (patch_embed): PatchEmbed(
        (proj): Conv3d(3, 1280, kernel_size=(2, 14, 14), stride=(2, 14, 14), bias=False)
      )
      (rotary_pos_emb): VisionRotaryEmbedding()
      (blocks): ModuleList(
        (0-31): 32 x Qwen2VLVisionBlock(
          (norm1): LayerNorm((1280,), eps=1e-06, elementwise_affine=True)
          (norm2): LayerNorm((1280,), eps=1e-06, elementwise_affine=True)
          (attn): VisionAttention(
            (qkv): Linear4bit(in_features=1280, out_features=3840, bias=True)
            (proj): Linear4bit(in_features=1280, out_features=1280, bias=True)
          )
          (mlp): VisionMlp(
            (fc1): Linear4bit(in_features=1280, out_features=5120, bias=True)
            (act): QuickGELUActivation()
            (fc2): Linear4bit(in_features=5120, out_features=1280, bias=True)
          )
        )
      )
      (merger): PatchMerger(
      

In [26]:
def collate_fn(examples):
    texts = [
        processor.apply_chat_template(example, tokenize = False) for example in examples
    ]
    image_inputs = [process_vision_info(example)[0] for example in examples]

    batch = processor(
        text = texts, images = image_inputs, return_tensors = "pt", padding = True
    )

    labels = batch["input_ids"].clone()
    labels[labels == processor.tokenizer.pad_token_id] = -100

    if isinstance(processor, Qwen2VLProcessor):
        image_tokens = [151652, 151653, 151655]
    else:
        image_tokens = [processor.tokenizer.convert_tokens_to_ids(processor.image_token)]

    for image_token_id in image_tokens:
        labels[labels == image_token_id] = -100

    batch["labels"] = labels

    return batch

In [27]:
training_args = SFTConfig(
    output_dir = "./qwen2-VL-2B-instruct-trl-VQA-MMStar3",
    num_train_epochs = 10,
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 8,
    gradient_checkpointing = True,
    optim = "adamw_torch",
    learning_rate = 2e-4,
    lr_scheduler_type = "constant",
    logging_steps = 10,
    eval_steps = 10,
    eval_strategy = "steps",
    save_strategy = "steps",
    save_steps = 10,
    greater_is_better = False,
    load_best_model_at_end = True,
    bf16 = True,
    tf32 = True,
    max_grad_norm = 0.3,
    warmup_ratio = 0.03,
    push_to_hub = False,
    report_to = "none",
    gradient_checkpointing_kwargs={"use_reentrant": False},
    dataset_text_field="",
    dataset_kwargs={"skip_prepare_dataset": True}
)
training_args.remove_unused_columns = False

In [28]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [29]:
from datasets import Dataset

trainer = SFTTrainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
    data_collator = collate_fn,
    peft_config = peft_config,
    processing_class = processor.tokenizer
)

trainer.train()

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss,Validation Loss
10,18.245500,1.979445
20,13.215500,1.442445
30,8.738500,0.904815
40,5.909700,0.774616
50,5.262100,0.749108
60,5.121700,0.728959
70,5.106900,0.712930
80,4.656600,0.692271
90,4.829500,0.669049
100,4.248500,0.627904


TrainOutput(global_step=220, training_loss=5.354431143673984, metrics={'train_runtime': 1438.8882, 'train_samples_per_second': 2.439, 'train_steps_per_second': 0.153, 'total_flos': 3.002839970747904e+16, 'train_loss': 5.354431143673984})

In [30]:
trainer.save_model(training_args.output_dir)